# Script to read Input Events Table & extract sedative data
- Erina Ghosh 2017/04/15

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline

## Reading InputEvents_mv

In [2]:
inp_evs_df = pd.read_csv('INPUTEVENTS_MV.csv', parse_dates=['STARTTIME', 'ENDTIME'])
inp_evs_df.shape

(3618991, 31)

In [3]:
inp_evs_df.head(3)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,TOTALAMOUNTUOM,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE
0,241,27063,139787,223259.0,2133-02-05 06:29:00,2133-02-05 08:45:00,225166,6.774532,mEq,NaN,...,ml,0,0,1,Rewritten,NaN,RN,2133-02-05 12:52:00,10.000000,0.050000
1,242,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 06:30:00,225944,28.132997,ml,30.142497,...,ml,0,0,0,FinishedRunning,NaN,NaN,NaN,28.132998,30.255817
2,243,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 06:30:00,225166,2.813300,mEq,NaN,...,ml,0,0,0,FinishedRunning,NaN,NaN,NaN,2.813300,0.050426


### Keeping selected columns:
    ROW_ID	INT
    SUBJECT_ID	INT
    HADM_ID	INT
    ICUSTAY_ID	INT
    STARTTIME	TIMESTAMP(0)
    ENDTIME	TIMESTAMP(0)
    ITEMID	INT
    AMOUNT	DOUBLE PRECISION
    AMOUNTUOM	VARCHAR(30)
    RATE	DOUBLE PRECISION
    RATEUOM
    ORDERID	BIGINT
    LINKORDERID	BIGINT
    ORDERCATEGORYNAME	VARCHAR(100)
    SECONDARYORDERCATEGORYNAME	VARCHAR(100)
    ORDERCOMPONENTTYPEDESCRIPTION	VARCHAR(200)
    ORDERCATEGORYDESCRIPTION	VARCHAR(50)
    PATIENTWEIGHT	DOUBLE PRECISION
    TOTALAMOUNT	DOUBLE PRECISION
    TOTALAMOUNTUOM	VARCHAR(50)
    STATUSDESCRIPTION	VARCHAR(30)

In [5]:
inp_evs_df = inp_evs_df[['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 
                             'STARTTIME', 'ENDTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 
                             'RATE', 'RATEUOM', 'ORDERID', 'LINKORDERID', 'ORDERCATEGORYNAME',
                             'SECONDARYORDERCATEGORYNAME', 'ORDERCOMPONENTTYPEDESCRIPTION', 
                             'ORDERCATEGORYDESCRIPTION', 'PATIENTWEIGHT', 'TOTALAMOUNT', 
                             'TOTALAMOUNTUOM', 'STATUSDESCRIPTION']].copy()
inp_evs_df.shape

(3618991, 21)

### Selecting rows with sedatives & neuromuscular blockers:
    221385,Lorazepam (Ativan)
    221623,Diazepam (Valium)
    221668,Midazolam (Versed)
    221744,Fentanyl
    221833,Hydromorphone (Dilaudid)
    222168,Propofol
    225972,Fentanyl (Push)
    225973,Meperidine (Demerol)
    225942,Fentanyl (Concentrate)
    225154,Morphine Sulfate
    225150,Dexmedetomidine (Precedex)
    221555,Cisatracurium
    222062,Vecuronium
    227520,Methadone Hydrochloride

In [12]:
med_list = pd.read_csv('pain_med_list.csv')
med_dict = pd.DataFrame.to_dict(med_list, orient='records')


In [20]:
med_dict={}
for i, r in med_list.iterrows():
    med_dict[r['itemid']] = r['label']


In [23]:
221384 in med_dict

False

In [27]:
(inp_evs_df['ITEMID'].iloc[0])

225166

In [31]:
inp_evs_df['sed_flag'] = 0
inp_evs_df['LABEL'] = np.NAN
for i, r in inp_evs_df.iterrows():
    if r['ITEMID'] in med_dict:
        inp_evs_df.ix[i, 'LABEL'] = med_dict[r['ITEMID']]
        inp_evs_df.ix[i, 'sed_flag'] = 1


KeyboardInterrupt: 

In [33]:
inp_evs_df.loc[inp_evs_df['ITEMID'] == 225973].shape

(596, 23)

In [34]:
inp_med_df = pd.DataFrame(columns= inp_evs_df.columns)
inp_med_df['LABEL'] = np.NAN
for k, v in med_dict.iteritems():
    med_df = inp_evs_df.loc[inp_evs_df['ITEMID'] == k]
    med_df['LABEL'] = v
    inp_med_df = pd.concat([inp_med_df, med_df])
inp_med_df.shape

/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(506114, 23)

### Number of rows with sedative/ neuromuscular blockers 

In [35]:
inp_med_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,ORDERCATEGORYNAME,SECONDARYORDERCATEGORYNAME,ORDERCOMPONENTTYPEDESCRIPTION,ORDERCATEGORYDESCRIPTION,PATIENTWEIGHT,TOTALAMOUNT,TOTALAMOUNTUOM,STATUSDESCRIPTION,sed_flag,LABEL
94387,95564.0,74835.0,133120.0,212772.0,2201-09-17 04:00:00,2201-09-18 20:00:00,227520.0,399.999984,mg,10.000000,...,01-Drips,02-Fluids (Crystalloids),Main order parameter,Continuous Med,100.9,500.0,ml,Rewritten,0.0,Methadone Hydrochloride
94407,94734.0,74835.0,133120.0,212772.0,2201-09-16 01:09:00,2201-09-16 06:08:00,227520.0,199.333331,mg,40.000000,...,01-Drips,02-Fluids (Crystalloids),Main order parameter,Continuous Med,100.9,250.0,ml,Rewritten,0.0,Methadone Hydrochloride
94437,96452.0,74835.0,133120.0,212772.0,2201-09-19 09:47:00,2201-09-20 12:04:00,227520.0,262.833323,mg,10.000000,...,01-Drips,02-Fluids (Crystalloids),Main order parameter,Continuous Med,100.9,500.0,ml,Stopped,0.0,Methadone Hydrochloride
94512,94613.0,74835.0,133120.0,212772.0,2201-09-16 12:01:00,2201-09-17 04:05:00,227520.0,160.666680,mg,10.000001,...,01-Drips,02-Fluids (Crystalloids),Main order parameter,Continuous Med,100.9,500.0,ml,Rewritten,0.0,Methadone Hydrochloride
94575,96760.0,74835.0,133120.0,212772.0,2201-09-17 17:47:00,2201-09-19 09:47:00,227520.0,399.999984,mg,10.000000,...,01-Drips,02-Fluids (Crystalloids),Main order parameter,Continuous Med,100.9,500.0,ml,Rewritten,0.0,Methadone Hydrochloride


## Filtering by cohort list

In [38]:
cohort_df = pd.read_csv('cohort2474.csv', parse_dates=['starttime', 'endtime', 'intime', 'outtime'])
cohort_df.shape

(2474, 17)

In [39]:
cohort_df.head(2)

,Unnamed: 0,subject_id,icustay_id,ventnum,starttime,endtime,duration_hours,age,rank,dayHrs,nightHrs,hadm_id,first_careunit,last_careunit,intime,outtime,los
0,1,10075,263099,1,2184-10-04 12:00:00,2184-10-08 16:00:00,100.0,62,1,52.0,48.0,153175,CSRU,CSRU,2184-10-04 09:03:19,2184-10-08 17:44:15,4.3618
1,2,10248,215903,1,2175-06-10 00:00:00,2175-06-13 16:00:00,88.0,69,1,45.0,43.0,108856,MICU,MICU,2175-06-09 18:32:51,2175-06-15 16:21:26,5.9087


Merging

In [41]:
inp_med_cohort = pd.merge(cohort_df[['hadm_id', 'starttime', 'endtime', 'age', 'dayHrs', 
                                     'nightHrs','intime', 'los', 'icustay_id']], inp_med_df, 
                          left_on = 'icustay_id', right_on= 'ICUSTAY_ID', how='inner')
inp_med_cohort.shape

(171508, 32)

Saving file with all meds for cohort

In [42]:
inp_med_cohort.to_csv('Cohort_all_med_170415.csv')